# Model - RandomForest with log of target

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *

In [3]:
df = pd.read_csv('../data/train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# ---Remove Outliers---

In [5]:
# OutlierRemover is a custom transformer imported from src/helpers.py
# It can remove outliers from multiple columns by specifying the iqr_multipler

# Features where there are significant outliers per EDA
outlier_cols = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea',
                'OpenPorchSF']

# Set the iqr multiplier to 5, to remove only the extreme outliers, in order to keep as much data as possible
outlier_remover = OutlierRemover(columns=outlier_cols,iqr_multiplier=5)

# Fit the dataset to the outlier remover and save to new variable
outlier_remover.fit(df)
df_proc = outlier_remover.transform(df)

# Check how many observations remain
df_proc.count()

Id               1428
MSSubClass       1428
MSZoning         1428
LotFrontage      1178
LotArea          1428
                 ... 
MoSold           1428
YrSold           1428
SaleType         1428
SaleCondition    1428
SalePrice        1428
Length: 81, dtype: int64

# ---Pipeline---

### 1. Filter for features to remove

In [6]:
# Defining the list of columns that will go into the pipeline
# Remove columns that should not be included in the pipeline

# Drop SalePrice b/c target feature
# Drop Id b/c no predictive meaning
# Drop Street b/c low variance
# Drop Utilities b/c low variance
# Drop GarageYrBlt b/c imputing is too speculative

all_columns = df.columns.tolist()
columns_remove_from_filter = ['SalePrice','Id','Street','Utilities','GarageYrBlt']

columns_to_keep = [item for item in all_columns if item not in columns_remove_from_filter]

In [7]:
# Preprocessor to filter unwanted columns, or unexpected columns.

preprocessor_filter = ColumnTransformer([
  ('passthrough','passthrough',columns_to_keep)
],remainder='drop', verbose_feature_names_out=False)

### 2. Imputing null variables

In [8]:
# Preprocessor for imputing all null variables

cols_impute_na = ['Alley', 'GarageType', 'Fence','BsmtQual', 'BsmtCond', 'BsmtExposure',
                  'BsmtFinType1','BsmtFinType2', 'FireplaceQu', 'GarageFinish', 'GarageQual','GarageCond',
                  'PoolQC', 'MiscFeature']
cols_impute_none = ['MasVnrType']
cols_impute_most_freq = ['Electrical','MSZoning','Exterior1st','Exterior2nd','KitchenQual','Functional']
cols_impute_zero = ['LotFrontage', 'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath',
                    'GarageCars','GarageArea']

preprocessor_imputer = ColumnTransformer([
  ('impute_na', SimpleImputer(strategy='constant', fill_value='NA'), cols_impute_na),
  ('impute_none', SimpleImputer(strategy='constant', fill_value='None'), cols_impute_none),
  ('impute_zero', SimpleImputer(strategy='constant', fill_value=0), cols_impute_zero),
  ('impute_most_freq', SimpleImputer(strategy='most_frequent'), cols_impute_most_freq)
], remainder='passthrough', verbose_feature_names_out=False)

### 3. Feature Engineering

In [9]:
# create a custom transformer to incorporate the engineered features into the pipeline

class FeatureEngineer(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    X = X.copy()

    X['RatioBathBed'] = (X['FullBath'].fillna(0)+X['HalfBath'].fillna(0)) / (X['BedroomAbvGr'].fillna(0)+1)
    X['HouseAge'] = X['YrSold'] - X['YearBuilt']
    X['TotalBaths'] = X['FullBath'].fillna(0) + (X['HalfBath'].fillna(0)*0.5) + X['BsmtFullBath'].fillna(0) + (X['BsmtHalfBath'].fillna(0)*0.5)
    X['FireBedRatio'] = X['Fireplaces'].fillna(0) / (X['BedroomAbvGr'].fillna(0) + 1)

    return X


engineer = FeatureEngineer()
df_engineered = engineer.transform(df_proc)

### 4. Mapping Ordinal Features

In [10]:
# Defining maps for Ordinal categories

dict_na_ex_6 = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
dict_na_gd_5 = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
dict_bsmt = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
dict_garage = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
dict_fence = {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}


ordinal_cat_map = [
  {'col':'BsmtQual','mapping':dict_na_ex_6},
  {'col':'BsmtCond','mapping':dict_na_ex_6},
  {'col':'BsmtExposure','mapping':dict_na_gd_5},
  {'col':'BsmtFinType1','mapping':dict_bsmt},
  {'col':'BsmtFinType2','mapping':dict_bsmt},
  {'col':'FireplaceQu','mapping':dict_na_ex_6},
  {'col':'GarageFinish','mapping':dict_garage},
  {'col':'GarageQual','mapping':dict_na_ex_6},
  {'col':'GarageCond','mapping':dict_na_ex_6},
  {'col':'PoolQC','mapping':dict_na_ex_6},
  {'col': 'ExterQual', 'mapping': dict_na_ex_6},
  {'col': 'ExterCond', 'mapping': dict_na_ex_6},
  {'col': 'HeatingQC', 'mapping': dict_na_ex_6},
  {'col': 'KitchenQual', 'mapping': dict_na_ex_6},
  {'col': 'Fence', 'mapping': dict_fence}
  ]

### 5. Encoding and Scaling

In [11]:
standard_cols = ['LotFrontage','GarageArea']

robust_cols = ['LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
               'GrLivArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MiscVal', 'LowQualFinSF',
               '3SsnPorch']

minmax_cols = ['YearBuilt','YearRemodAdd']


preprocessor_scalers = ColumnTransformer([
  ('standard_scaler', StandardScaler(), standard_cols),
  ('robust_scaler', RobustScaler(), robust_cols),
  ('minmax_scaler', MinMaxScaler(), minmax_cols)
], remainder='passthrough', verbose_feature_names_out=False)

scaler_pipe = Pipeline([
  ('preprocessor_scalers', preprocessor_scalers),
])


In [12]:
# Preprocessor for encoding and scaling

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
           'PoolQC','KitchenQual','Fence']


preprocessor_encode_scale = ColumnTransformer([
  ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols),
  ('oe', OrdinalEncoder(mapping=ordinal_cat_map), oe_cols)],
  remainder=scaler_pipe, verbose_feature_names_out=False)

# --- Train/Val Split, Fit & Transform ---

In [13]:
# Train/Val split, target feature is SalePrice

X = df.drop(columns='SalePrice')
y = df['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# take the log of the target to normalize the distribution

log_y_train = np.log(y_train)
log_y_val = np.log(y_val)

In [15]:
# Main pipeline, include all preprocessors

main_pipe = Pipeline([
  ('preprocessor_filter', preprocessor_filter),
  ('preprocessor_imputer', preprocessor_imputer),
  ('engineer', engineer),
  ('preprocessor_encode_scale', preprocessor_encode_scale)
])

In [16]:
from sklearn.ensemble import RandomForestRegressor

# creating a pipeline for the RandomeForest
rf_pipe = Pipeline([
  ('main_pipe', main_pipe),
  ('rf', RandomForestRegressor(random_state=42))
])

In [17]:
# rf_pipe.get_params()

# --- Model Evaluation ---

In [18]:
from sklearn.model_selection import RandomizedSearchCV

params = {
  'rf__n_estimators': [100, 200, 300,500],
  'rf__max_depth': [5, 10, 15, 20, None],
  'rf__min_samples_split': [2, 5, 10, 20],
  'rf__min_samples_leaf': [1, 2, 4, 8],
  'rf__max_features': ['sqrt','log2',0.33, 0.5, 0.75]
}

rf_gs = RandomizedSearchCV(rf_pipe, params, n_iter=20, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42, verbose=1)

rf_gs.fit(X_train, log_y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/annahan/Documents/GitHub/learning-fuze/live_class_W1D2_0125/.conda/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('main_pipe',
                                              Pipeline(steps=[('preprocessor_filter',
                                                               ColumnTransformer(transformers=[('passthrough',
                                                                                                'passthrough',
                                                                                                ['MSSubClass',
                                                                                                 'MSZoning',
                                                                                                 'LotFrontage',
                                                                                                 'LotArea',
                                                                                                 'Alley',
                                                                                                 'LotShape',
                                                                                                 'LandContour',
                                                                                                 'LotConfig',
                                                                                                 'LandSlope',
                                                                                                 'Neighborhood',
                                                                                                 'Condition1',
                                                                                                 'Condition2',
                                                                                                 'BldgType',
                                                                                                 'HouseStyle',
                                                                                                 'OverallQual',
                                                                                                 'O...
                                                                                 verbose_feature_names_out=False))])),
                                             ('rf',
                                              RandomForestRegressor(random_state=42))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'rf__max_depth': [5, 10, 15, 20, None],
                                        'rf__max_features': ['sqrt', 'log2',
                                                             0.33, 0.5, 0.75],
                                        'rf__min_samples_leaf': [1, 2, 4, 8],
                                        'rf__min_samples_split': [2, 5, 10, 20],
                                        'rf__n_estimators': [100, 200, 300,
                                                             500]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [19]:
rf_gs.best_params_

{'rf__n_estimators': 300,
 'rf__min_samples_split': 2,
 'rf__min_samples_leaf': 1,
 'rf__max_features': 0.5,
 'rf__max_depth': 10}

In [20]:
results_train = evaluate_regression(rf_gs, X_train, log_y_train, 'rf_gs')
results_val = evaluate_regression(rf_gs, X_val, log_y_val, 'rf_gs')

In [21]:
results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
rf_gs,0.043571,0.003598,12839.339678,0.003641,0.976397,0.97466


In [22]:
results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
rf_gs,0.094434,0.020754,29198.297527,0.007956,0.888784,0.846616


In [23]:
# predict val
log_val_preds = rf_gs.predict(X_val)

# convert prediction back by using exponential
val_preds = np.exp(log_val_preds)

# calculate log RMSE
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.1440641858202628

# --- Predicting test ---

In [24]:
# import the test data

test_data = pd.read_csv('../data/test.csv')

In [25]:
# save the id column to be added back later to the predictions

test_id = test_data.loc[:,'Id'].copy()
test_id = pd.DataFrame(test_id)

In [27]:
# predict test (keeping in mind it is still log)

log_test_pred = rf_gs.predict(test_data)
log_test_pred = pd.DataFrame(log_test_pred)

In [ ]:
# convert the test predictions back using exponential

test_pred = np.exp(log_test_pred)

In [ ]:
# adding the id's back to the test predictions

combined_test = test_id.join(test_pred)

In [ ]:
# change the column names to match the submission format

combined_test.columns = ['Id','SalePrice']

In [ ]:
# check the final submission

combined_test

,Id,SalePrice
0,1461,123299.437500
1,1462,160231.203125
2,1463,184277.125000
3,1464,188793.453125
4,1465,195872.796875
...,...,...
1454,2915,86462.984375
1455,2916,86881.890625
1456,2917,159923.281250
1457,2918,124269.921875


In [ ]:
# save results to csv

# combined_test.to_csv('../data/test_preds5_lasso_log.csv', index=False)

In [ ]:
# save results to csv - submission

# combined_test.to_csv('../data/submission5.csv', index=False)

In [ ]:
# calculate kaggle ranking, what percentage I am in

1557/3639

0.42786479802143446